In [18]:
# Enviroment setup
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import io
from PIL import Image
from skimage.metrics import structural_similarity as ssim

class ImageCompressionEnv(gym.Env):
    def __init__(self, images):
        super(ImageCompressionEnv, self).__init__()
        self.images = images
        self.current_index = 0
        self.current_image = self.images[self.current_index]
        
        # Define action and observation space
        self.action_space = spaces.Discrete(101)  # Compression levels [0, 100]
        self.observation_space = spaces.Box(low=0, high=255, shape=self.current_image.shape, dtype=np.uint8)

    def step(self, action):
        compressed_image = self.compress_image(self.current_image, action)
        reward = ssim(self.current_image, compressed_image)
        self.current_index = (self.current_index + 1) % len(self.images)
        self.current_image = self.images[self.current_index]
        done = self.current_index == 0
        return compressed_image, reward, done, {}

    def reset(self):
        self.current_index = 0
        self.current_image = self.images[self.current_index]
        return self.current_image

    def compress_image(self, image : np.ndarray, compression_level):
        print(image.shape)
        image = Image.fromarray(image)
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG", quality=compression_level)
        buffer.seek(0)
        r = buffer.getvalue()
        r = Image.open(io.BytesIO(r))
        r = np.array(r)
        print(r.shape)
        return r

# Example usage
images = [np.random.randint(0, 256, (64, 64, 3), dtype=np.uint8) for _ in range(100)]
env = ImageCompressionEnv(images)


In [19]:
# PPO algorithm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

class PPO(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PPO, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.policy_layer = nn.Linear(128, output_dim)
        self.value_layer = nn.Linear(128, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        policy = self.policy_layer(x)
        value = self.value_layer(x)
        return policy, value

    def get_action(self, state):
        policy, _ = self.forward(state)
        policy_dist = Categorical(logits=policy)
        action = policy_dist.sample()
        return action.item(), policy_dist.log_prob(action), policy_dist.entropy()

    def evaluate_action(self, state, action):
        policy, value = self.forward(state)
        policy_dist = Categorical(logits=policy)
        action_log_probs = policy_dist.log_prob(action)
        dist_entropy = policy_dist.entropy()
        return action_log_probs, torch.squeeze(value), dist_entropy

def compute_gae(rewards, masks, values, gamma=0.99, tau=0.95):
    returns = []
    gae = 0
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

def ppo_update(agent, optimizer, trajectories, clip_param=0.2):
    states = torch.stack([trajectory[0] for trajectory in trajectories])
    actions = torch.tensor([trajectory[1] for trajectory in trajectories])
    log_probs_old = torch.tensor([trajectory[2] for trajectory in trajectories])
    returns = torch.tensor([trajectory[3] for trajectory in trajectories])
    advantages = returns - torch.tensor([trajectory[4] for trajectory in trajectories])
    
    for _ in range(4):
        log_probs, state_values, dist_entropy = agent.evaluate_action(states, actions)
        ratio = torch.exp(log_probs - log_probs_old)
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()
        value_loss = (returns - state_values).pow(2).mean()
        loss = policy_loss + 0.5 * value_loss - 0.01 * dist_entropy.mean()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Example usage
input_dim = 64 * 64 * 3  # Example image size
output_dim = 101  # Compression levels from 0 to 100
agent = PPO(input_dim, output_dim)
optimizer = optim.Adam(agent.parameters(), lr=3e-4)


In [20]:
# Training Loop
def train(env, agent, optimizer, num_episodes=1000, gamma=0.99, clip_param=0.2):
    all_rewards = []
    for episode in range(num_episodes):
        state = env.reset()
        state = torch.FloatTensor(state.flatten()).unsqueeze(0)
        episode_reward = 0
        done = False
        trajectories = []

        while not done:
            action, log_prob, _ = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            next_state = torch.FloatTensor(next_state.flatten()).unsqueeze(0)

            mask = 1 if not done else 0
            trajectories.append((state, action, log_prob, reward, mask))

            state = next_state
            episode_reward += reward

        all_rewards.append(episode_reward)

        states = torch.stack([trajectory[0] for trajectory in trajectories])
        actions = torch.tensor([trajectory[1] for trajectory in trajectories])
        log_probs = torch.tensor([trajectory[2] for trajectory in trajectories])
        rewards = torch.tensor([trajectory[3] for trajectory in trajectories])
        masks = torch.tensor([trajectory[4] for trajectory in trajectories])

        with torch.no_grad():
            _, next_value = agent(states[-1])
            values = torch.cat([agent(states)[1], next_value.unsqueeze(0)])

        returns = compute_gae(rewards, masks, values, gamma)
        ppo_update(agent, optimizer, trajectories, clip_param)

        if episode % 10 == 0:
            print(f"Episode {episode}, Reward: {episode_reward}")

    return all_rewards

# Example usage
train(env, agent, optimizer)


(64, 64, 3)
(64, 64, 3)


ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.

In [ ]:
# Evaluation
def evaluate(env, agent, num_episodes=100):
    all_rewards = []
    for _ in range(num_episodes):
        state = env.reset()
        state = torch.FloatTensor(state.flatten()).unsqueeze(0)
        episode_reward = 0
        done = False

        while not done:
            action, _, _ = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            next_state = torch.FloatTensor(next_state.flatten()).unsqueeze(0)
            state = next_state
            episode_reward += reward

        all_rewards.append(episode_reward)
    return all_rewards

# Example usage
evaluation_rewards = evaluate(env, agent)
print(f"Average evaluation reward: {np.mean(evaluation_rewards)}")
